In [1]:
import glob, json, os, sys
sys.path.append('/root/SWE-bench')
sys.path.append('/root/SWE-bench/swebench/metrics') # TODO: Replace with path to `SWE-bench/swebench/metrics` folder
from conversion import convert_log_to_ground_truth
from getters import get_logs_gold
from monitor import monitor_validation, monitor_logs_same_diff
sys.path = sys.path[:-1]

sys.path.append('/root/SWE-bench/swebench/harness') # TODO: Replace with path to `SWE-bench/swebench/harness` folder
from utils import has_attribute_or_import_error
sys.path = sys.path[:-1]

/opt/conda/envs/swe-bench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Declare repository; Fetch tasks, logs

In [2]:
# import glob 

# logs_list = glob.glob("/mnt/llm/home/ibragim-bad/data/swe_large/log_dir/*.log") + glob.glob("/mnt/llm/home/ibragim-bad/data/swe_large_new/log_dir/*.json")
# inst = [l.split('/')[-1].split('.')[0] for l in  logs_list]
# with open("/root/SWE-bench/swe_like_111k.json") as f:
#     tasks = json.load(f)
# cut_tasks = [t for t in tasks if t['instance_id'] not in set(inst)]

In [3]:
# from collections import defaultdict

# ct_dict = defaultdict(list)

# for t in cut_tasks:
#     ct_dict[t['repo']].append(t)

# mx_len = max([len(v) for v in ct_dict.values()])
# new_ct = []
# for i in range(mx_len):
#     for k in ct_dict:
#         if i < len(ct_dict[k]):
#             t = ct_dict[k][i]
#             new_ct.append(t)

In [4]:
# N = 500



# for i in range(0, len(new_ct), N):
#     with open(f"/root/data/swe/tasks_{i//N}.json", "w") as f:
#         json.dump(new_ct[i:i+N], f)

In [5]:
lst = open("/root/data/progress_2.txt").readlines()
lst = set([l.strip() for l in lst])

from collections import defaultdict

tasks = []
tasks_dict = defaultdict(set)
for l in lst:
    with open(f"/root/data/swe/{l}") as f:
        cur_t =  json.load(f)
        tasks +=cur_t
        for t in cur_t:
            tasks_dict[l.split('/')[0]].add(t['instance_id'])

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f1692b3ebb0>>
Traceback (most recent call last):
  File "/opt/conda/envs/swe-bench/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
# import json
# l = {'swebench': {'resolved': {'count': 7, 'turns': {'avg': 9.571428571428571, 'median': 9.0, 'p90': 13.4}, 'opened_files': {'avg_precision': 0.8571428571428571, 'avg_recall': 1.0, 'avg_jaccard_similarity': 0.8571428571428571}, 'edit_loop_fraction': 0.14285714285714285, 'exit_cost_fraction': 0.0, 'patched_files': {'avg_precision': 0.9285714285714286, 'avg_recall': 1.0, 'avg_jaccard_similarity': 0.9285714285714286}}, 'failed': {'count': 88, 'turns': {'avg': 22.545454545454547, 'median': 16.5, 'p90': 45.3}, 'opened_files': {'avg_precision': 0.3901515151515151, 'avg_recall': 0.4431818181818182, 'avg_jaccard_similarity': 0.3901515151515151}, 'edit_loop_fraction': 0.25, 'exit_cost_fraction': 0.0, 'patched_files': {'avg_precision': 0.369047619047619, 'avg_recall': 0.45454545454545453, 'avg_jaccard_similarity': 0.369047619047619}}, 'skipped': {'count': 191, 'turns': {'avg': 39.02094240837696, 'median': 30.0, 'p90': 82.0}, 'opened_files': {'avg_precision': 0.22168638516806055, 'avg_recall': 0.31413612565445026, 'avg_jaccard_similarity': 0.22168638516806055}, 'edit_loop_fraction': 0.35602094240837695, 'exit_cost_fraction': 0.0}, 'ignored': {'count': 6}, 'total': {'count': 292}}}
# print(json.dumps(l, indent=4))

In [3]:
repo = 'pydicom/pydicom' # TODO: Replace with repository name
log_dir = '/root/data/log_dir_2' # TODO: Replace with path to folder of execution logs

Get map of version to setup commit

In [7]:
# tasks = json.load(open('path/to/pydicom-task-instances.json')) # TODO: Replace with path to versioned candidate task instances
for t in tasks:
    t["version"] = "0.0"


tasks = sorted(tasks, key=lambda x: x['created_at'], reverse=True)

version_to_setup_commit = {}
for t in tasks:
    if 'version' in t and t['version'] not in version_to_setup_commit:
        version_to_setup_commit[t['version']] = t['base_commit']
assert(
    sorted(list([x or "" for x in version_to_setup_commit.keys()])) ==
    sorted(list(set([t['version'] or "" for t in tasks if 'version' in t])))
)
tasks = {t['instance_id']: t for t in tasks}

#### Monitor Validation

In [7]:
failed_install, corrupt_test_patch, corrupt_patch, timeout, success = monitor_validation(log_dir)

Total Attempts: 77492
Failed: 48903
Usable: 28589
Corrupt Test: 385
Corrupt Diff: 225
Test Script Timeout: 309
Success E2E: 27670


In [9]:
fs = failed_install + success
fs = set([f.split('/')[-1].split('.')[0] for f in fs if f])

co = {}
for t in tasks_dict:
    co[t] = len(tasks_dict[t] & fs)

In [10]:
nums = []
for c in sorted(co):
    if co[c] == 0:
        nums.append(c.split('_')[1].split('.')[0])

In [11]:
' '.join((nums))

''

In [18]:
len([f for f in failed_install if '0.0' not in f]) + len(success)

67850

In [8]:
timeouts = 0
timeout_installs = []
for fp in failed_install:
    with open(fp) as f:
        l = f.read()
        if '>>>>> Init Timed Out' in l:
            timeouts+=1
            timeout_installs.append(fp.split('/')[-1].split('.')[0])

In [4]:
with open("/root/SWE-bench/swe_like_111k.json") as f:
    tasks = json.load(f)

In [9]:
hst = set(timeout_installs)
timeout_tasks = [t for t in tasks if t['instance_id'] in hst]
len(timeout_tasks)

11956

In [18]:
same = [fp.split('/')[-1].split('.')[0] for fp in logs_same]

In [19]:
N = 500

hss = set(same)
same_tasks = [t for t in tasks if t['instance_id'] in hss]
len(same_tasks)


for i in range(0, len(same_tasks), N):
    with open(f"/root/data/swe/same_{i//N}.json", "w") as f:
        json.dump(same_tasks[i:i+N], f)

In [11]:
N = 500



for i in range(0, len(timeout_tasks), N):
    with open(f"/root/data/swe/timeouts_{i//N}.json", "w") as f:
        json.dump(timeout_tasks[i:i+N], f)

In [12]:
timeout_tasks[0]

{'base_commit': 'b501c92485576298b7ef46e7ab64b188b0f8ab3d',
 'created_at': '2021-02-10T16:06:41Z',
 'hints_text': '',
 'instance_id': 'geoscixyz__geosci-labs-32',
 'issue_numbers': [30],
 'meta': {'failed_lite_validators': ['has_short_problem_statement',
   'has_removed_files'],
  'has_test_patch': True,
  'is_lite': False},
 'patch': 'diff --git a/.travis.yml b/.travis.yml\ndeleted file mode 100644\nindex 45e7082..0000000\n--- a/.travis.yml\n+++ /dev/null\n@@ -1,56 +0,0 @@\n-language: python\n-python:\n-  - 3.6\n-\n-sudo: false\n-\n-env:\n-  global:\n-    - PYPI_PY=3.6  # deploy to pypi from python 3.6\n-    - TWINE_USERNAME=lheagy\n-\n-env:\n-  - TEST=test_dcip.py\n-  - TEST=test_em.py\n-  - TEST=test_gpr.py\n-  - TEST=test_inversion.py\n-  - TEST=test_mag.py\n-  - TEST=test_seismic.py\n-  - TEST=test_gravity.py\n-  - TEST=style\n-\n-# Setup anaconda\n-before_install:\n-# Install packages\n-  - wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh

In [13]:
# import os 

# fails = set([os.path.basename(t).split('.')[0] for t in failed_install if '0.0' not in t])

# for t in sorted(tasks_dict):
#     print(t, len(tasks_dict[t]), len(tasks_dict[t] & fails))

# tasks_dict['tasks_0.json'] & fails


In [13]:
logs_same, logs_diff = monitor_logs_same_diff(log_dir)
print(f"Logs same: {len(logs_same)}")
print(f"Logs diff: {len(logs_diff)}")

Logs same: 21425
Logs diff: 6297


In [23]:
import re

# Regex pattern to extract the module name
pattern = r"(?<=No module named ').*?(?=')"

def extract_module_name(error_message):
    # Extracting the module name
    match = re.search(pattern, error_message)
    # Check if a match is found
    if match:
        return match.group(0)
    return None


In [24]:
module_names = []
for fp in logs_same:
    with open(fp) as f:
        l = f.read()
        if "ModuleNotFoundError" in l:
            module_names.append(extract_module_name(l))

In [31]:
from collections import Counter

Counter(module_names).most_common(50)

[('responses', 184),
 ('mock', 177),
 ('hypothesis', 129),
 ('django', 125),
 ('freezegun', 104),
 ('_pytest._io', 97),
 ('jsonschema', 97),
 ('trustme', 75),
 ('sqlalchemy', 74),
 ('numpy', 72),
 ('PIL', 70),
 ('betamax', 60),
 ('vcr', 60),
 ('requests_mock', 56),
 (None, 54),
 ('docker', 51),
 ('tensorflow', 50),
 ('pytz', 44),
 ('requests', 43),
 ('pint.unit', 40),
 ('torch', 40),
 ('fastapi', 40),
 ('webtest', 38),
 ('_distutils_hack', 37),
 ('yaml', 37),
 ('attr', 37),
 ('lxml', 35),
 ('scripttest', 32),
 ('coverage_conditional_plugin', 29),
 ('pandas.tseries.tools', 27),
 ('asynctest', 26),
 ('httpretty', 24),
 ('pyspark', 24),
 ('pytest_recording', 24),
 ('jinja2', 24),
 ('jsonschema.compat', 22),
 ('mpi4py', 21),
 ('click', 18),
 ('pytest_timeout', 18),
 ('installer', 17),
 ('tvm', 17),
 ('dask.compatibility', 16),
 ('astropy.tests.pytest_plugins', 16),
 ('pkg_resources.extern', 15),
 ('selenium', 15),
 ('jax.config', 15),
 ('psycopg2', 15),
 ('sphinx', 14),
 ('azure', 14),
 ('

In [17]:
logs_same[-10:]

['/root/data/log_dir_2/jborchma__qtrade-7.log',
 '/root/data/log_dir_2/nolar__kopf-826.log',
 '/root/data/log_dir_2/OpenMDAO__dymos-240.log',
 '/root/data/log_dir_2/DataBiosphere__toil-3460.log',
 '/root/data/log_dir_2/dlint-py__dlint-49.log',
 '/root/data/log_dir_2/charettes__django-seal-36.log',
 '/root/data/log_dir_2/arviz-devs__arviz-557.log',
 '/root/data/log_dir_2/ros-infrastructure__bloom-614.log',
 '/root/data/log_dir_2/dpkp__kafka-python-1024.log',
 '/root/data/log_dir_2/mmagnuski__borsar-73.log']

In [15]:
len(tasks)

34000

#### Get [FP]2[FP] Tests

In [16]:
inst_id_to_gt = {}
for d in logs_diff:
    status_gt = convert_log_to_ground_truth(d[0])
    inst_id_to_gt[d[0]] = status_gt

#### Create Task Instances `.json` file

In [17]:
tasks_final = []
get_id_from_log = lambda x: x.split('/')[-1].split('.')[0]
for k, v in inst_id_to_gt.items():
    if len(v['FAIL_TO_PASS']) == 0:
        continue
    if not get_id_from_log(k) in tasks:
        continue
    task = tasks[get_id_from_log(k)]
    task['FAIL_TO_PASS'] = v['FAIL_TO_PASS']
    task['PASS_TO_PASS'] = v['PASS_TO_PASS']
    task['environment_setup_commit'] = version_to_setup_commit["0.0"]

    # Do not consider tasks where the log before the patch has an attribute/import error
    log_path = os.path.join(log_dir, f'{task["instance_id"]}.log')
    log_before, log_after = get_logs_gold(log_path)
    if has_attribute_or_import_error(log_before):
        continue

    tasks_final.append(task)
print(f"Final number of task instances: ", len(tasks_final))

# SAVE_PATH = "path/to/save/tasks/to.json" # TODO: Replace this with a path to a .json file to save the task instances
# with open(SAVE_PATH, 'w') as f:
#     json.dump(tasks_final, fp=f)

Final number of task instances:  1929
